# Setup cloud resouces

You need `gcloud` installed, initialized and logged in with an account that has access to the `deadwood-d4a4b` project.

## Workload Identity federation

This is necessary, that the Github action can use the associated user credentials to request a google could access token in exchange for a Github openID token. The access is done via a google service accout that has the permission to access google artifact registry

```
export PROJECT_ID="deadwood-d4a4d"
gcloud init
```

Create a service account with the right permissions:

`github-build-container-artifac@deadwood-d4a4b.iam.gserviceaccount.com`

`export SERVICE_ACCOUNT="github-build-container-artifac@deadwood-d4a4b.iam.gserviceaccount.com"`

create the federation pool

`gcloud iam workload-identity-pools create "github-action" --location="europe-west3"  --display-name="GitHub Actions"`

get the full id for the pool - Github will need that
`gcloud iam workload-identity-pools describe "github-action"  --location="global" --format="value(name)"`

gcloud iam workload-identity-pools describe "github-action"  --location="global" --format="value(name)"

set that as a variable
`export WORKLOAD_IDENTITY_POOL_ID="projects/20668345835/locations/global/workloadIdentityPools/github-action"`


Now we need the provider for OpenID in the created pool

`gcloud iam workload-identity-pools providers create-oidc "github" --location="global" --workload-identity-pool="github-action" --display-name="Github provider" --attribute-mapping="google.subject=assertion.sub,attribute.actor=assertion.actor,attribute.repository=assertion.repository" --issuer-uri="https://token.actions.githubusercontent.com"`

next, we need to allow the repo

`export REPO="hydrocode-de/deadwood-scripts"`

`gcloud iam service-accounts add-iam-policy-binding "${SERVICE_ACCOUNT}" --role="roles/iam.workloadIdentityUser" --member="principalSet://iam.googleapis.com/${WORKLOAD_IDENTITY_POOL_ID}/attribute.repository/${REPO}"`

For the Github action the Provider id is needed:

that is added to the github secrets

`gcloud iam workload-identity-pools providers describe "github" --location="global" --workload-identity-pool="github-action" --format="value(name)"`